In [ ]:
print()

In [ ]:
from experiments.attn.victory import *

In [ ]:
losses, rate = run_trial(common.ConvModel, boardsize=9, n_layers=20, D=16)

In [ ]:
from experiments.architecture import *

In [ ]:
run('fc-conv', 256, 16, 16*1024, 8e-4)

In [ ]:
%debug

In [ ]:
242/32